# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに向けて、未知のエンティティを特定するための言語モデルを開発し、改善することを目的としています。具体的には、Qwen 2 7b - Instructモデルを使用し、質問者、回答者、推測者の役割を持つエージェントを設計しています。

### 問題の取り組み
このノートブックは、言語モデルにおいて複雑な2対2のチーム戦を効果的にプレイするために、効率的な情報収集と推理が求められる「20の質問」ゲームの戦略を検討しています。特に、ルールに基づいて正しい推測を行い、エージェント同士が協力しながら勝利を目指すという挑戦が提示されています。

### 手法とライブラリ
- **使用モデル**: 主にQwen 2 7b - Instructを使用しており、Hugging Faceの他のモデルへの切り替えも可能です。
- **戦略**: 
  - 質問者エージェントが初期質問を行い、その後はゲーム履歴に基づいて質問を生成します。
  - 回答者エージェントは、過去の質問と回答を考慮しつつ、最も多数の意見を基にした回答を生成します。
  - 推測者エージェントはゲーム履歴を参照して推測を行います。
- **ライブラリ**: `transformers`, `torch`, `huggingface_hub`などが使用され、これによりモデルのトークナイザーや重みをロードしています。また、量子化を行うために`BitsAndBytes`ライブラリも利用されています。

### コードの構成
- **依存関係のインストール**: `accelerate`と`bitsandbytes`をインストールしています。
- **モデルのダウンロード**: `huggingface_hub`からモデルをダウンロードし、適切な形式で保存します。
- **メインスクリプトの作成**: Kaggle環境内で動作するエージェントのインターフェースを作成し、契約されたメモリ制限内でモデルを使用するための設定が含まれています。
- **エージェントの初期化と実行**: 観察オブジェクトを用いて、ゲームの各ターンにおける質問、回答、推測を処理します。また、モデルの生成テキストとそのパラメータ調整も行っています。

### 改善点
- より堅牢なモデルの試験、異なるルールや戦略の追加、ゲームの状態追跡機能の実装などが提案されています。これにより、モデルの性能向上が期待できます。

このノートブックは、言語モデルが「20の質問」という複雑なタスクで効果的に機能するための土台を築くことを目指し、モデルの改善と戦術的な応答生成に焦点を当てています。

---


# 用語概説 
以下は、Jupyter Notebook内で使われている専門用語の解説です。初心者の方がつまずきやすいが、ある程度の知識を有する方に向けた解説です。

1. **オーバーフィットモデル**:
   - モデルがトレーニングデータに対して過剰に適合し、新しいデータや汎用性に苦しむ状況を指します。特に公のリーダーボードでのパフォーマンスは良好でも、普遍性がない場合に問題が発生します。

2. **量子化 (Quantization)**:
   - モデルのパラメータ（例:重み）を低いビット数で表現することにより、メモリ使用量と計算の効率を向上させる手法です。例として、4ビットや8ビットの量子化があります。

3. **bitsandbytes**:
   - 大規模なトランスフォーマーモデルを効率的にメモリにロードするためのライブラリです。特に、量子化技術を利用して大きなモデルをサポートします。

4. **T4 GPU**:
   - 特にディープラーニング向けに設計されたNVIDIAのGPUの一種です。推論やデータ処理を高速化するために広く使用されます。

5. **MAX_NEW_TOKENS**:
   - テキスト生成において新たに生成されるトークンの最大数を指定するパラメータです。この数が多いほど長い出力が期待されます。

6. **温度 (Temperature)**:
   - テキスト生成におけるランダム性を制御するパラメータです。低い値は決定的な出力を生成し、高い値は創造的で多様な出力を促します。

7. **デバッグ (Debug)**:
   - プログラムの不具合や動作を確認するプロセスです。このNotebook内では、コード実行の各段階で情報を表示し、問題点を明らかにするために用いられています。

8. **自己対戦 (Self-play)**:
   - 同じエージェントが自分自身と対戦することで、性能向上や最適化を行う手法です。このコンペティションでは、エージェントが自己対戦を通じて評価されます。

9. **ハードコーディング (Hardcoding)**:
   - プログラム内部に固定値やコードを直接記述することです。柔軟性には欠けるが、特定の値を使う場合に簡易な実装が可能になります。

10. **スナップショット (Snapshot)**:
    - モデルやその重みの状態を特定の時点で「スナップショット」として保存し、後に復元または使用できる形です。モデルのデプロイや評価時に便利です。

11. **エージェント (Agent)**:
    - 指定された役割（質問者、回答者、推測者など）を持つプログラム部分を指し、この場合は游戏の状況に応じて行動します。

これらの用語は初心者にとって特に重要ですが、実務での経験が浅いために直感的に理解しにくい部分もあります。注意深く取り扱うことが求められます。

---


# はじめに
このノートブックは提出の準備が整っており、改善戦略や検討すべきアイデアに関する詳しい指示が含まれています。モデルはQwen 2 7b - Instructに設定されていますが、huggingfaceの他のモデルに変更することも可能です。

このノートブックに変更を加えなければ、すべてのセルを実行するかノートブックをコミットしてコンペティションに提出できます。

## このノートブックはどのようなスコアになりますか？

これは難しい質問です。キーワードの検索空間は広大で、未知のエンティティの特定は言語モデルにとって難しいです。また、このコンペティションでは、異なるエージェントチームの中で協力してプレイするという複雑さが加わります。この2対2の設定は、さまざまなゲームの結果を引き起こす挑戦的な文脈を作り出します。この件について多くの議論がされてきました。その上、キーワードリストに依存して推測を生成するオーバーフィットモデルもあります。これらは、公のリーダーボードではポイントを獲得しますが、エージェントに秘密のキーワードリストが与えられると失敗します。これにより、公のリーダーボードはモデルの性能の良いベンチマークにならない可能性があります。自己評価によってモデルの性能を判断することをお勧めします。
1. 質問は論理的に思われますか？
2. 推測は質問に従っていますか？
3. あなたのエージェントは適切に質問に答えていますか？
4. 変更を加えた場合、以前よりもモデルは良くなりましたか？

これらの質問への回答はスコアやメダルを与えるものではありませんが、私たちが解決しようとしている真の問題の核心にあります：

**小さな言語モデルを開発し、未知のエンティティ特定ゲームをプレイし、そのスキルを向上させることです。**

## 提出スクリプト

以下のスクリプトは、20の質問ゲーム用にKaggle環境内で実行される.pyファイルを作成します。このスクリプトは、各ラウンドでテキストを生成するために（以下からダウンロードされた）LLMを利用します。スクリプトには、ゲーム環境内で提供される構造と変数と相互作用するクラスが含まれています。また、質問者、回答者、推測者の役割を取るためにLLMを促す3つの関数も含まれています。最後に、初期化されたモデルとトークナイザーを使用したテキスト生成、およびゲームからの適切なエージェントコールが含まれています。

### 戦略

このスクリプトで利用される戦略には以下が含まれます：
* ゲームスタートを助け、LLMにゲームの進め方を示すための「ハードコーディングされた」最初の質問。
* ゲーム履歴をレビューして応答を作成する質問者エージェントと推測者エージェント。
* 5回ループして最も頻繁な応答を返す回答者エージェント。
* コンペティションの制限をシミュレーションするためにcuda:0にモデルをロード。

スクリプトには含まれていない：
* 公のリーダーボードでのパフォーマンスを向上させるためにLLMにキーワードを供給すること。
* 推測者が検索するための堅牢なキーワードリストを提供する外部単語リストの供給。 


## 改善点
* より堅牢なLLMモデルを追加。huggingfaceのオープンLLMリーダーボードで約7Bまたはそれ以下のモデルをレビューし、それらをスクリプトにロードしてみてください。特定のモデルのロードに問題があるでしょうが、試行錯誤して問題解決を楽しむことができます。
* Phi3、Gemma、またはLLaMa 3を試して、Gwenと比較してどうなるか確認してください。（提出フォルダにライブラリをダウンロードしておくことを忘れずに！）
* より大きなモデルを追加する場合、bitsandbytesとaccelerateをインストールおよびインポートし、quantization-configを4ビットに設定してモデルをロードしてください。
* 提供された回答に基づいて、キーワードがどのカテゴリに属する可能性が高いかを定義するゲーム状態追跡を追加してください。
* 初期の広範なゲーム質問から、後半の具体的な質問に移行するためのゲームフェーズ追跡を追加してください。
* 直行テキスト生成の代わりにマルチステッププロンプトを追加してください。
* temperatureなどのテキスト生成パラメータを試してみてください。
* インストラクションチューニングされたQwenモデルは、時折マルチレベルプロンプトに苦労することがあります。他のモデルは自己対話に優れているでしょうか？
* 微調整されたモデル。標準のモデルは20の質問のプレイに特化していません。いくつかは他のモデルよりも優れています。進行中の20の質問コンペティションからのQ&Aペアのデータセットが役立つかもしれませんか？

## よくある質問
* メモリエラーが表示されますか？ファクトリーリセットを行い、スクリプトを再実行してください。おそらくモデルを二重にロードしてしまったのです！
* 質問者または推測者からのいくつかの応答が適切にフォーマットされていないのはなぜですか？応答をパースするための創造的な方法を見つける必要があります。これはプロンプトを使用するか、生成されたテキストから有効な応答を抽出するためにハードコーディングされたパーサを使用することで実現できます。

# コードに進みましょう

### 最初 - 依存関係のインストール
スクリプト用の依存関係をいくつかインストールし、提出時にアップロードされる一時ディレクトリにもインストールします。これは、スクリプトがKaggle環境内で正常に実行されるために必須です。
他のパッケージをインストールする必要がある場合は、下記の両方のセルに追加してください。 


In [ ]:
!pip install accelerate bitsandbytes

このセルは少し時間がかかります。 


In [ ]:
import os
os.system("pip install -t /tmp/submission/lib accelerate bitsandbytes")

### スナップショットダウンロード：このセルは何をしますか？

このセクションでは、モデルを1つの提出フォルダに圧縮（zip）した後、提出後にスクリプトによって呼び出される一時ディレクトリにダウンロードして保存します。ゲーム環境はKaggle環境で実行されるため、モデルの重みはダウンロードして提出しなければなりません。このセルの読み込みには数分かかります。

注：これはモデルやトークナイザーを読み込むわけではありません。そのステップはメインスクリプトで行われます。

現在、モデルはQwen 2 7b Instructに設定されています。 


In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

model_path = Path("/tmp/submission/")
if model_path.exists():
    shutil.rmtree(model_path)
model_path.mkdir(parents=True)

snapshot_download(
    repo_id= "Qwen/Qwen2-7B-Instruct",
    local_dir=model_path
)

# メインスクリプト 

ノートブックの最上部のマジックコマンドは、Kaggle環境内で実行され、コンペティションに提出される.pyファイルを作成します。

- このスクリプトを評価のために実行すると、量子化されたモデルがGPUメモリにロードされます。Qwen 2 7bは大きいため、4ビットの量子化バージョンがロードされ、15GBのうち7.4GBのメモリを使用します。 


In [ ]:
%%writefile main.py

# 上記の行は評価セッションを実行する場合はコメントアウトしてください。

import os

#このスクリプトがどの環境で実行されているかを確認します。これにより、異なる環境で提出と評価を行うことができます。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if not os.path.exists(KAGGLE_AGENT_PATH):
    KAGGLE_AGENT_PATH = "/tmp/submission/"

import sys
import itertools
import re
import torch
import typing as t
from pathlib import Path
from pydantic import BaseModel, Field, field_validator
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from IPython.display import display, Markdown

model_initialized = False
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t_dtype = torch.bfloat16
DEBUG = False

#==================量子化設定================
# Gwen 2 7bを量子化する必要があります。それは、コンペティションで使用される単一のT4 GPUに収まらないからです。
# これは、より大きなモデルを量子化するための構成がどのように見えるかを参照するために含まれています。
# また、モデルが既に読み込まれているか確認する条件文も展開しています。もしそれが真であれば、OOMエラーを防ぐために読み込みをスキップします。
# これは、スクリプトの修正、評価セッションの確認、新しい試みを行う際に非常に役立ちます。

if 'model' in locals() and 'tokenizer' in locals() and model is not None and tokenizer is not None:
    print("モデルとトークナイザーはすでに存在します。作成処理をスキップします。")
else:
    quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                             bnb_4bit_compute_dtype=t_dtype)
    
    model = AutoModelForCausalLM.from_pretrained(KAGGLE_AGENT_PATH, 
                                                 device_map=device, 
                                                 torch_dtype=t_dtype,
                                                 quantization_config=quantization_config)
    
    tokenizer = AutoTokenizer.from_pretrained(KAGGLE_AGENT_PATH, 
                                              torch_dtype=t_dtype)

        
#==================KAGGLE環境用の観察クラス================
# このクラスは、Kaggle環境からのobs入力をこのスクリプトで使用できるようにします。
# 観察クラスはTeam Riggingの公開ノートブックに感謝します。

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_text(self, *, skip_guesses: bool = False, skip_answer: bool = False, skip_question: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""{'**質問:** ' + question if not skip_question else ''} -> {'**回答:** ' + answer if not skip_answer else ''}
            {'**以前の推測:** ' + guess if not skip_guesses else ''}
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "まだなし."


# =================システムプロンプト================
# ここにあなたのシステムプロンプトがあり、テキスト生成中のメッセージテンプレートに挿入されます。
# 指示/チャットモデルの場合、モデルに従うべきコンテキストと方向性を提供します。
# ここで展開できるさまざまなプロンプト戦略がいくつかあります。
# ゲームの状態やフェーズ（初期/中期/後期）に応じて異なるシステムプロンプトを持つことも可能です。

system_prompt = """あなたは、インタラクティブな未知のエンティティ特定ゲーム「20の質問」をプレイしています。
あなたは質問者、回答者、推測者の全役割に熟練した熟練者です。
あなたの目標は、できるだけ少ないターンで秘密のキーワードを推測して勝利することです。"""

# =================質問者エージェント================
# 質問者エージェントは最初にキーワードが場所かどうかを尋ね、その後赤道の北側に位置しているかどうかを尋ねます。ここではLLMは呼び出されません。
# キーワードが「場所」でない場合、それは原因が無生物であるかどうかを尋ねます。
# これによりゲーム環境の少数のサンプルをLLMに提供し、最初のカテゴリを二分するのに役立ちます。
# それ以降（step > 1）、前の履歴に基づいて生成された質問を詢います。
# 追加の条件付き質問をLLMを呼び出す前に加え、探索空間をさらに絞り込むことができます。
# LLMを呼び出す前に、チェーンオブシンク推論や他のテクニックを追加するためのテキスト生成ステップを追加することもできます。
# max_new_tokensやtemperatureパラメータをチェックすることを忘れずに。
# プロンプトの複雑さが増すにつれて、異なるパラメータを持つ異なるgenerate_text関数を作成したいでしょう。
# たとえば、generate_longははるかに高い新しいトークン制限と高い温度を持ちます。

def ask(observation: Observation) -> str:
    if observation.step == 0:
        return "秘密のキーワードは場所ですか？"
    elif observation.step == 1:
        if len(observation.answers) > 0 and observation.answers[0] == "yes":
            return "都市ですか？"
        else:
            return "生きているものですか？"
    game_history = observation.get_history_text(skip_guesses=True)

    try:
        think = generate_text(f"""
        あなたは質問者として次の質問をしており、以下のことを知っています：

====== ゲーム履歴 =====
{game_history}
=========================

        上記の履歴に基づき、次にどのように質問を進めるべきかを説明してください。
        """)
        
        parse = generate_text(f"""
        現在の情報を確認し、次のベストな質問を教えてください。
        あまり情報がない場合は、検索空間からカテゴリを排除するためにはい/いいえの質問をするのがベストです。

        概要: "{think}""
        
        質問をただ聞いてください。質問と疑問符以外のテキストを提供しないでください。
        """
        ).lower()
        if DEBUG:
            display(Markdown(f"### デバッグ: 質問者の思考:"))
            display(Markdown(think))
            display(Markdown(f"### デバッグ: 質問者のパース:"))
            display(Markdown(parse))
        return parse
    except Exception as e:
        print(f"質問を尋ねる際のエラー: {e}")
        return '場所ですか？'
    

# =================回答者エージェント================
# 5回ループし、最も頻繁な応答を取ります。
# これはTeam Riggingのノートブックで使用されたテクニックです。
# 単一の応答よりもわずかに効果的です。

def answer(observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("キーワードが回答者に提供されませんでした", file=sys.stderr)
        return "yes"
            
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            response = generate_text(f"""
            20の質問ゲーム。このキーワードに対してはい/いいえで答えてください: {observation.keyword} 

            質問: "{last_question}"
            
            ルール:
            1. あなたのキーワード: {observation.keyword} にのみ注意してください。これはカテゴリに分けられます: {observation.category}.
            2. 'はい'または'いいえ'だけを答えてください。その他の情報は必要ありません。
            ここにあなたの回答を入力してください:
            """
            ).lower()
            if DEBUG:
                display(Markdown(f"### デバッグ: 回答者の応答:"))
                display(Markdown(f"DEBUG: {response}"))
            
            yes_no = re.findall(r'\b(yes|no)\b', response)
            if yes_no:
                responses.append(yes_no[0])

        if DEBUG:
            display(Markdown(f"DEBUG: すべての回答者の応答 {responses}"))
        return max(set(responses), key=responses.count)

    except Exception as e:
        print(f"回答者でエラー: {e}", file=sys.stderr)
        return "yes"

# ================推測者エージェント================
# このエージェントはゲーム履歴を使用して推測を行います。
# 一部の場合、LLMが関連する単語ループにハマるため、これはあまり役に立たないことがあります。
# 異なるプロンプト戦略を試すことや、推測をスキップすることが良いでしょう。
# パース戦略を使用すると、ゲームの応答が有効で理解可能であることを保証します。

def guess(observation: Observation) -> str:
    game_history = observation.get_history_text(skip_guesses=False)
    
    think = generate_text(f"""
    あなたは現在、推測者としてキーワードの情報に基づいた推測を行います。

== ゲーム履歴 ==
{game_history}
==================

    提案された関連する1つのカテゴリと、可能な推測を提案してください。
    あなたの根拠を説明してください。
    """
    )
    
    parse = generate_text(f"""
    以下の応答を確認し、次のベストな推測を教えてください。
    あまり情報がない場合は、野生の具体的な推測をしてください。
    質問のように推測を尋ねず、単語またはフレーズの推測を直接提供してください。

    概要: "{think}"

    あなたの推測はこちら:
    """
    ).lower()
    if DEBUG:
        print(f"ゲーム履歴: {game_history}")
        display(Markdown(f"### デバッグ: 推測者の思考:"))
        display(Markdown(think))
        display(Markdown(f"### デバッグ: 推測者のパース:"))
        display(Markdown(parse))
    
    return parse

#==============LLMメッセージ生成===============
# 変更したいパラメータがいくつかあります。
# temperatureは生成されるテキストのランダム性を設定します。
# 値の低い方がより決定論的で、高い方がより創造的です。
# max_new_tokensは生成されるテキストの長さを設定します。

    
def generate_text(prompt:str) -> str:
    sys_prompt = system_prompt
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt},
    ]
    text = tokenizer.apply_chat_template(messages, 
                                         tokenize=False, 
                                         add_generation_prompt=True)
    
    inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(inputs.input_ids,
                                   max_new_tokens=350, 
                                   do_sample=True, 
                                   temperature=0.1) #これは生成テキストのランダム性/創造性を設定します。低いほど決定論的です。
    
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

#================KAGGLE環境用エージェントの初期化================
# 評価中にモデルと対戦してエージェントの1つを'input'に変更すると楽しさが増します！
# ボーナスとして、微調整データセットを作成する可能性が開かれます！素晴らしいですね！
# 評価セッションを微調整データセットの作成セッションにすることを検討できます。これにより、出力をデータフレームに保存できます。
# 無限の可能性がありますが、時間に制約があります。  

agent = None

def observe(obs: t.Any) -> str:
    global agent
    observation = Observation(**obs.__dict__)

    try:
        match observation.turnType:
            case "ask":
                agent = ask(observation)
                return agent
            case "answer":
                agent = answer(observation)
                return agent
            case "guess":
                agent = guess(observation)
                return agent

            case _:
                raise ValueError("不明なターンタイプ")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    return observe(obs)

# =========================TORCHバックエンドを有効化===========================
# このノートブックでスクリプトを実行するときにエラーが発生することがあります。
# これらのバックエンド設定は、エラーなしでスクリプトが実行されることを保証します。やったね！

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# 提出 - タールボール
このインストールと圧縮は、kaggleが期待するフォーマットに提出物を作成します。モデル、スクリプト、および提出フォルダを含めて圧縮します。pipインストールをスクリプトに追加する場合は、それらを提出tarballにも追加する必要があります。提出tarballに追加するために/tmp/submission/libにインストールしてください。


In [ ]:
!apt install pigz pv

In [ ]:
!tar --use-compress-program='pigz --fast' \
     -cf submission.tar.gz \
     -C /kaggle/working main.py \
     -C /tmp/submission/ \
     .

# 評価
perform_evalがTrueに設定されている場合、以下のコードはKaggleリポジトリから最新の公的キーワードをダウンロードし、ランダムなキーワードを生成します。これにより、Kaggle環境の初期化を避けられ、メモリエラーを回避できます。デバッグモードを有効にするかどうかが求められます。これは、プロンプト構造を変更したり、プロンプト内のオブジェクトを呼び出したり、マルチステッププロンプト構造で最終応答がゲーム内にない場合に役立ちます。

* 評価を実行する際は、必ずアクセラレーターをオンにしてください。 
* 評価セッションを実行する場合、プロンプト構造を変えて異なる戦略を試したくなることがあります。その場合、LLM（モデルとトークナイザー）の初期化を行うスクリプトのトップをコメントアウトすることをお勧めします。そうしないと、毎回モデルを読み込んでしまい、メモリ不足になります。
* 異なるモデルを試しているときは、応答が得られるまでにかかる時間を確認してください。コンペティションは応答時間を60秒に制限しています！（最初にGPU加速が有効化されていることを確認してください）。

### perform_eval? これは何のためですか？
perform_evalをTrueに設定すると、すべてのセルを実行したりノートブックをコミットしたりすると、エージェントがKaggleリポジトリのキーワードを使用してノートブック内で自己対戦する20の質問ゲームで評価されます。やった！

これをTrueに設定すると、デバッグモードかどうかを聞かれるため、約2〜3分後にプロンプトが表示されます。

評価セッションをスキップしたい場合は、Falseに設定して迅速なコミットと提出を行ってください。
#### **評価の迅速化のためにGPU加速を必ずオンにしてください。**


In [ ]:
perform_eval = False

In [ ]:
if perform_eval:
    import requests
    import json
    import re
    import typing as t
    import random
    import time
    from IPython.display import display, Markdown
    import signal

    # ===============KAGGLE GITHUBから最新のキーワードを取得================
    # これはモデルがキーワードを探すためにトレーニングされることはありません。
    # これはただ評価に使用されるリストを取得するだけです。
    
    url = "https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py"
    response = requests.get(url)

    if response.status_code == 200:
        match = re.search(r'KEYWORDS_JSON = """(.*?)"""', response.text, re.DOTALL)
        if match:
            json_str = match.group(1)
            keywords_dict = json.loads(json_str)
        else:
            print("ファイル内でKEYWORDS_JSON変数を見つけることができませんでした。")
    else:
        print("リクエストが失敗しました。ステータスコード:", response.status_code)
        import time

    def select_random_keyword(keywords_dict: t.List[dict]) -> str:
        category = random.choice(keywords_dict)
        keyword_dict = random.choice(category['words'])
        return keyword_dict['keyword'].lower()
    
    #===============20の質問評価セッション=====================
    def input_timeout(prompt, timeout):
        def timeout_handler(signum, frame):
            raise TimeoutError("入力タイムアウト: {}秒後".format(timeout))

        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)

        try:
            user_input = input(prompt)
            signal.alarm(0)
            return user_input
        except TimeoutError as e:
            print(e)
            return None

    timeout = 10

    try:
        DEBUG_input = input_timeout("詳細なデバッグモードを有効にしますか？ (y/n) [デフォルトはn] ", timeout)
        DEBUG = DEBUG_input.lower() == 'y' if DEBUG_input else False
    except:
        DEBUG = False
        print("入力が受け取られませんでした。デフォルトでfalseに設定します。")

    class MockObservation:
        def __init__(self, step: int, role: str, turnType: str, keyword: str, category: str, questions: list[str], answers: list[str], guesses: list[str]):
            self.step = step
            self.role = role
            self.turnType = turnType
            self.keyword = keyword
            self.category = category
            self.questions = questions
            self.answers = answers
            self.guesses = guesses

    def test_20_questions():
        global DEBUG
        step = 0
        role = "answerer"
        turnType = "ask"
        keyword = select_random_keyword(keywords_dict)
        category = ""
        questions = []
        answers = []
        guesses = []
        display(Markdown("# 20の質問評価ゲームを開始します..."))
        display(Markdown(f"### **キーワード:** {keyword}"))

        for i in range(60):
            obs = MockObservation(step, role, turnType, keyword, category, questions, answers, guesses)

            start_time = time.time()
            response = agent_fn(obs, None)
            end_time = time.time()

            response_time = end_time - start_time
            if response_time > 60:
                display(Markdown(f"**警告:** 応答時間が長すぎます。このゲームから失格になる可能性があります: {response_time:.2f}秒。右側のパネルでセッションオプションを用いてGPU加速が有効になっていることを確認してください。"))
                break

            # 適切なリストに応答を記録
            if turnType == 'ask':
                questions.append(response)
                turnType = 'answer'
            elif turnType == 'answer':
                answers.append(response)
                turnType = 'guess'
            elif turnType == 'guess':
                guesses.append(response)
                if response.lower() == keyword.lower():
                    display(Markdown(f"## **キーワード '{keyword}'が正しく推測されました！ ゲームを終了します。**"))
                    break
                turnType = 'ask'
                step += 1

            display(Markdown(f"ステップ {step} | 応答: {response} | {response_time:.2f}秒"))
        display(Markdown(f"最終質問: {', '.join(questions)}"))
        display(Markdown(f"最終回答: {', '.join(answers)}"))
        display(Markdown(f"最終推測: {', '.join(guesses)}"))

    # テストを実行
    test_20_questions()

---

# コメント 

> ## Pang Luo
> 
> 本当に印象的な作業です。「考える-答える」アプローチは素晴らしいです。エージェントは『ダヌーブ』を推測しましたが、正しい答えは『ドニエプル』でした。とても近いです。
> 
> 
> 


---

> ## Thuận Đoàn Vũ
> 
> あなたの素晴らしい仕事に感謝します。1つ質問があります。なぜ4ビットではなく8ビットでモデルをロードしないのですか？8ビットの方が良くないですか？
> 
> 
> 
> > ## Matthew S Farmer 投稿者
> > 
> > 良い洞察ですね。試してみてください！このノートブックはモジュール式に作られているので、改善を行い、テストすることができます。🤗
> > 
> > 
> > 
> > ## Matthew S Farmer 投稿者
> > 
> > その質問に技術的に答えると、低精度はパフォーマンスが低くなるという意見があります。それは小さなモデルにはあまり重要ではありませんが、依然として存在します。しかし、このコンペティションでは、計算制約と応答ごとの時間制限もあります。テスト時には、割り当てられたVRAMと応答時間制約内に収めながら、モデルの能力を最大限に引き出すことを確認して下さい。 
> > 
> > 


---

> ## Marília Prata
> 
> 素晴らしいノートブックと貴重なアドバイス（改善点、よくある質問、マークダウンセル）。
> 
> 
> > ## Matthew S Farmer 投稿者
> > 
> > それはあなたからの高い評価です、Marilia。ありがとう！ 
> > 
> > 
